In [2]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

%matplotlib inline
import matplotlib.pyplot as plt

img_width, img_height = 299,299
train_data_dir = "train"
validation_data_dir = "val"
batch_size = 16
epochs = 50

# model = applications.resnet50.ResNet50(weights='imagenet', include_top=False, input_shape = (img_width, img_height, 3))

model = applications.xception.Xception(include_top=True, weights='imagenet', classes=1000)
model.summary()

# for layer in model.layers:
#     layer.trainable = False

# #Adding custom Layers 
# x = model.output
# x = Flatten()(x)
# x = Dense(1000, activation="relu")(x)
# x = Dropout(0.5)(x)
# predictions = Dense(2, activation="softmax")(x)

# # creating the final model 
# model_final = Model(inputs = model.input, outputs = predictions)

# # compile the model 
# model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])
# model_final.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
block1_conv1 (Conv2D)            (None, None, None, 32 864         input_2[0][0]                    
____________________________________________________________________________________________________
block1_conv1_bn (BatchNormalizat (None, None, None, 32 128         block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_conv1_act (Activation)    (None, None, None, 32 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [10]:
model.layers[13].output

<tf.Tensor 'block2_pool_1/MaxPool:0' shape=(?, ?, ?, 128) dtype=float32>

In [7]:
train_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

test_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical")

validation_generator = test_datagen.flow_from_directory(
validation_data_dir,
target_size = (img_height, img_width),
class_mode = "categorical")

nb_train_samples = train_generator.samples
nb_validation_samples = validation_generator.samples

# Save the model according to the conditions  
checkpoint = ModelCheckpoint("resnet50_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')


Found 50133 images belonging to 2 classes.
Found 9044 images belonging to 2 classes.


In [8]:

# Train the model 
model_final.fit_generator(
train_generator,
samples_per_epoch = nb_train_samples,
epochs = epochs,
steps_per_epoch = int(nb_train_samples / batch_size),
validation_data = validation_generator,
nb_val_samples = nb_validation_samples,
callbacks = [checkpoint, early])

/Users/kenneth/.pyenv/versions/3.6.1/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  # Remove the CWD from sys.path while we load stuff.
/Users/kenneth/.pyenv/versions/3.6.1/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=50, steps_per_epoch=3133, validation_data=<keras.pre..., callbacks=[<keras.ca..., validation_steps=9044)`
  # Remove the CWD from sys.path while we load stuff.


Epoch 1/50
  11/3133 [..............................] - ETA: 11051s - loss: 0.9564 - acc: 0.3977

KeyboardInterrupt: 